# Chloropleth visualization

### Imports

In [1]:
import cartopy.crs as ccrs
import cartopy as ctpy
import matplotlib.pyplot as plt

from cartopy.feature import ShapelyFeature
from matplotlib.colors import Normalize
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.cm import coolwarm

from read_data import *
from create_product import *

### Constants

In [2]:
OUTPUT_PATH = "out"

### Loading COVID Data

In [3]:
data, timesteps = get_derived_covid_active_change_global()

print(data)

    iso                                              shape  1/22/20  1/23/20  \
0    AF  POLYGON ((66.52226562500002 37.348486328125, 6...        0        0   
1    AL  POLYGON ((19.34238281250001 41.869091796875, 1...        0        0   
2    DZ  POLYGON ((8.576562500000023 36.93720703125, 8....        0        0   
3    AD  POLYGON ((1.7060546875 42.50332031249999, 1.67...        0        0   
4    AO  (POLYGON ((13.07275390625 -4.634765625, 13.057...        0        0   
..   ..                                                ...      ...      ...   
185  VN  (POLYGON ((104.06396484375 10.3908203125, 104....        0        2   
186  PS  (POLYGON ((34.47734375000002 31.58486328124999...        0        0   
187  YE  (POLYGON ((53.08564453125001 16.648388671875, ...        0        0   
188  ZM  POLYGON ((30.39609375000001 -15.64306640625, 3...        0        0   
189  ZW  POLYGON ((31.28789062500002 -22.40205078125001...        0        0   

     1/24/20  1/25/20  1/26/20  1/27/20

### Cloropleth

In [75]:
def get_data_max_val(data, timesteps):
    max_val = 0
    for t in timesteps:
        col_max = data[t].max()
        if col_max > max_val:
            max_val= col_max
    return max_val

def get_data_min_val(data, timesteps):
    min_val = 0
    for t in timesteps:
        col_min = data[t].min()
        if col_min < min_val:
            min_val = col_min
    return min_val

# https://stackoverflow.com/a/20528097/7759262
def get_shifted_cmap(cmap, start=0, midpoint=0.5, stop=1.0):
    cdict = {
        'red': [],
        'green': [],
        'blue': [],
        'alpha': []
    }
    reg_index = np.linspace(start, stop, 257)
    shift_index = np.hstack([
        np.linspace(0.0, midpoint, 128, endpoint=False), 
        np.linspace(midpoint, 1.0, 129, endpoint=True)
    ])
    for ri, si in zip(reg_index, shift_index):
        r, g, b, a = cmap(ri)
        cdict['red'].append((si, r, r))
        cdict['green'].append((si, g, g))
        cdict['blue'].append((si, b, b))
        cdict['alpha'].append((si, a, a))
    newcmap = LinearSegmentedColormap("newColormap", cdict)
    return newcmap

def get_color(val, min_val, max_val):
    n_levels = max_val-min_val
    ctr = 1 - max_val / (max_val + abs(min_val))
    val_norm = (val - min_val) / n_levels
    cmap = get_shifted_cmap(coolwarm, midpoint=ctr)
    return cmap(val_norm)

def cloropleth(data, timesteps, step):
    """
    Creates a cloropleth figure for the given timestep "step"
    """
    time = timesteps[step]
    shapes = data['shape']
    vals = data[time]
    max_vals = data[timesteps].max(axis=1)
    min_vals = data[timesteps].min(axis=1)
    
    # Create the figure
    fig = plt.figure(figsize=(16,8), dpi=90)
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_extent([-180,180,-90,90])
    ax.coastlines()
    
    # This masks the ocean
    ax.add_feature(ctpy.feature.OCEAN, zorder=99, edgecolor='k', color="white")
    ax.add_feature(ctpy.feature.BORDERS, zorder=99, edgecolor='black')
    ax.add_feature(ctpy.feature.COASTLINE, zorder=99, edgecolor='black')
    ax.set_global()
        
    # Add country coloring
    for i in range(shapes.shape[0]):
        shape = shapes[i]
        val = vals[i]
        max_val = max_vals[i]
        min_val = min_vals[i]
        color = get_color(val, min_val, max_val)
        shape_feature = ShapelyFeature([shape], ccrs.PlateCarree(), facecolor=color)
        ax.add_feature(shape_feature)
    
    plt.text(-170,78, time, backgroundcolor="black", color="white", fontsize="xx-large", zorder=100, transform=ccrs.PlateCarree())

    return fig

### Generate sample for testing

In [84]:
test_imgs = []
test_timesteps = timesteps[-10:]
for t in range(len(test_timesteps)):
    fig = cloropleth(data, timesteps, t)
    fig.savefig(f"{OUTPUT_PATH}/chloropleth_test_{t}.png", format='png')
    img = plt_to_img(fig)
    test_imgs.append(img)

    # Preserve memory by removing figures. No need to retain them
    plt.clf()
    plt.close(fig)
    
imgs_to_video(test_imgs, f"{OUTPUT_PATH}/chloropleth_test_ideo.avi", framerate=1)

### Generate images

In [85]:
imgs = []
for t in range(len(timesteps)):
    fig = cloropleth(data, timesteps, t)
    fig.savefig(f"{OUTPUT_PATH}/chloropleth_{t}.png", format='png')
    img = plt_to_img(fig)
    imgs.append(img)

    # Preserve memory by removing figures. No need to retain them
    plt.clf()
    plt.close(fig)

### Generate video

In [1]:
imgs_to_video(imgs, f"{OUTPUT_PATH}/chloropleth_video.avi", framerate=15)

NameError: name 'imgs_to_video' is not defined